In [ ]:
import numpy as np
import pandas as pd 

from tyssue.solvers import QSSolver
from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.basic_events import reconnect
from tyssue.dynamics import model_factory, effectors
from tyssue.geometry.planar_geometry import AnnularGeometry
from tyssue.utils import modify_segments

from tyssue.draw import sheet_view

from CellPacking.tissuegeneration import generate_ellipsis, EllipsisGeometry
from CellPacking.plot import superimpose_sheet_view


import matplotlib.pyplot as plt

In [ ]:
def model_init(sheet, specs, modifiers):
    
    sheet.update_specs(specs, reset=True)
    modify_segments(sheet, modifiers)

In [ ]:
a = 10
b = 4
cell_height = 1.5
barrier_height = 2
ellipsis = generate_ellipsis(Nf=150, a=a, b=b, cell_height=cell_height, apical='out') 

specs = {'settings':{'lumen_prefered_vol': np.pi*a*b*1.2,
                    'lumen_volume': np.pi*a*b,
                    'lumen_vol': np.pi*a*b,
                    'lumen_elasticity':3e-6, 
                    'geometry':'flat',
                    'barrier_height':barrier_height, 
                    'inside':'basal'},
         
        'vert':{ 'basal_shift':0,
               'barrier_elasticity':100,
               'is_active':1},
         
        'face':{'prefered_area':0.8,
                 'area_elasticity':10,
                 'is_alive':1,},
         
        'edge': {#'line_tension':1,
                  'is_active':1},
                  }

modifiers = {
    'apical': {
        'edge': {'line_tension': 1, # gamma
                 }
        },
    'basal': {
        'edge': {'line_tension': 1, # beta
                 }
        },
    'lateral': {
        'edge': {'line_tension': 1/2,  # alpha (accounting for double half edges)
                 }
        },
    }



model_init(ellipsis, specs, modifiers)

ellipsis.scoords = ["s" + c for c in ellipsis.coords]
ellipsis.tcoords = ["t" + c for c in ellipsis.coords]

EllipsisGeometry.update_all(ellipsis)

In [ ]:
ellipsis_init = ellipsis.copy(deep_copy=True)

In [ ]:
fig, ax = sheet_view(ellipsis_init)
fig.set_size_inches((20,20))

In [ ]:
# Solver
solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)


from CellPacking.tissuegeneration import BarrierElasticity
# Model
model = model_factory(
    [
        BarrierElasticity,
        effectors.FaceAreaElasticity,
#         effectors.LineTension,
        effectors.LumenAreaElasticity,
    ])

# ellipsis.vert_df[["x", "y"]] += np.random.normal(scale=0.1, size=(ellipsis.Nv, 2))
# EllipsisGeometry.update_all(ellipsis)

res = solver_qs.find_energy_min(ellipsis, EllipsisGeometry, model, periodic=False, options={"gtol": 1e-2})
print (res.success)

In [ ]:
from tyssue.draw.plt_draw import quick_edge_draw
fig = superimpose_sheet_view(ellipsis_init, ellipsis, ['init', 'equilibrium'])

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig

In [ ]:
from tyssue.draw.plt_draw import plot_forces, plot_scaled_energies
ellipsis_init.scoords = ["s" + c for c in ellipsis_init.coords]
ellipsis_init.tcoords = ["t" + c for c in ellipsis_init.coords]

fig, ax = plot_forces(ellipsis_init, EllipsisGeometry, model, list('xy'), 0.1 )

theta = np.arctan((ellipsis_init.vert_df.x / ellipsis_init.vert_df.y))
ax.plot((a+barrier_height)*np.cos(theta), (b+barrier_height)*np.sin(theta),'.', markersize=50)


fig.set_size_inches((100,100))

In [ ]:
from tyssue.draw.plt_draw import plot_forces, plot_scaled_energies
fig, ax = plot_forces(ellipsis, EllipsisGeometry, model, list('xy'), 0.1 )


# position de la barriere
theta = np.arctan((ellipsis.vert_df.x / ellipsis.vert_df.y))
ellipsis.vert_df['theta2'] = np.arctan((ellipsis.vert_df.y / ellipsis.vert_df.x))
ax.plot((a+barrier_height)*np.cos(theta), (b+barrier_height)*np.sin(theta),'.', markersize=50)

x = ellipsis.vert_df[ellipsis.vert_df["delta_rho"]>0]["x"]
y = ellipsis.vert_df[ellipsis.vert_df["delta_rho"]>0]["y"]
ax.plot(x,y, '.', markersize=50)

# ax.plot(ellipsis.vert_df.barrier_rho*np.cos(t), ellipsis.vert_df.barrier_rho*np.sin(t),'.', markersize=20)

# ax.plot(ellipsis.vert_df.x*np.cos(t), ellipsis.vert_df.y*np.sin(t), '.', markersize=20)

fig.set_size_inches((100,100))

In [ ]:
plt.hist(ellipsis.edge_df[ellipsis.edge_df['segment']=='lateral']['length'], alpha=0.5)
plt.hist(ellipsis_init.edge_df[ellipsis_init.edge_df['segment']=='lateral']['length'], alpha=0.5)

In [ ]:
plt.hist(ellipsis.face_df['area'],alpha=0.5)
plt.hist(ellipsis_init.face_df['area'],alpha=0.5)

In [ ]:
scales = np.linspace(0.8, 1.2, 30)
fig, ax = plot_scaled_energies(ellipsis, EllipsisGeometry, model,
                               scales, ax=None)
fig.set_size_inches((10,10))

In [ ]:
ellipsis.edge_df[['sx','sy','tx','ty','rx', 'ry', 'dx','dy','fx','fy', 'nz']]

In [ ]:
import numpy as np
from shapely.geometry.polygon import LinearRing, LineString

def ellipse_polyline(ellipses, n=100):
    t = np.linspace(0, 2*np.pi, n, endpoint=False)
    st = np.sin(t)
    ct = np.cos(t)
    result = []
    for x0, y0, a, b, angle in ellipses:
        angle = np.deg2rad(angle)
        sa = np.sin(angle)
        ca = np.cos(angle)
        p = np.empty((n, 2))
        p[:, 0] = x0 + a * ca * ct - b * sa * st
        p[:, 1] = y0 + a * sa * ct + b * ca * st
        result.append(p)
    return result

def intersections(a, b):
    ea = LinearRing(a)
    eb = LinearRing(b)
    mp = ea.intersection(eb)
    if type(mp)==LineString:
        return None, None
    x = [p.x for p in mp]
    y = [p.y for p in mp]
    return x, y



# U, S, V = np.linalg.svd(np.stack((ellipsis.vert_df[ellipsis.vert_df['segment']=='apical']['x'], 
#                                  ellipsis.vert_df[ellipsis.vert_df['segment']=='apical']['y'])))
# tt = np.linspace(0, 2*np.pi, 1000)
# circle = np.stack((np.cos(tt), np.sin(tt)))  
# transform = np.sqrt(2/300) * U.dot(np.diag(S))
# fit = transform.dot(circle)



for i in np.arange(0.1, 3, 0.01):
    ellipses = [(0, 0, ellipsis.vert_df['x'].max(), ellipsis.vert_df['y'].max(), 0), (ellipsis.vert_df.loc[20]['x'], ellipsis.vert_df.loc[20]['y'], i, i, 0)]
#     ellipses = [(0, 0, fit[0:].max(), fit[1:].max(), 0), (ellipsis.vert_df.loc[0]['x'], ellipsis.vert_df.loc[0]['y'], i, i, 0)]
    a, b = ellipse_polyline(ellipses)
    x, y = intersections(a, b)
    if x is not None:
        break
print(x,y)

fig, ax = plot_forces(ellipsis, EllipsisGeometry, model, list('xy'), 0.1 )
ax.plot(x, y, "o")
ax.plot(ellipsis.vert_df.loc[20]['x'], ellipsis.vert_df.loc[20]['y'],'.', markersize=20)
ax.plot(a[:,0], a[:,1])
ax.plot(b[:,0], b[:,1])

ax.set_aspect('equal')
fig.set_size_inches((50,50))